# パスデータの作成

In [1]:
# import libraries

import numpy as np
import pandas as pd
import gc
import os
import json
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import plotly.express as px
import re
import datetime as dt
import japanize_matplotlib
import itertools
year_pattern = r'([1-2][0-9]{3})'

In [2]:
DATAPATH = "../../../data/"

In [3]:
def get_metadata(filename):
    with open(DATAPATH + filename, encoding="utf-8", mode='r') as f:
        for line in f:
            yield line

In [4]:
with open(DATAPATH + "DBLP/keywords.txt") as f:
    keywords = list(map(lambda x: x.replace("\n", ""), f.readlines()))
key_sets = set(keywords)
len(keywords), keywords[0]

(1233, 'radiometry')

In [5]:
word_dict = {}
for num, kw in enumerate(keywords):
    word_dict[kw] = num

In [6]:
N = len(keywords)
START_YEAR = 2000
END_YEAR   = 2020
YEAR_STD   = END_YEAR - START_YEAR 

In [7]:
%%time
datas = np.zeros((N,N,YEAR_STD+1))
print(datas.shape)

metadata = get_metadata("dblpv13.txt")
for paper in tqdm(metadata):
    data = json.loads(paper)
    try:
        k = data.get('keywords')
        y = data.get('year')
        if len(k) == 0 or len(k) == 1:
            pass
        else:
            if y >= START_YEAR and END_YEAR >= y:
                k = list(map(lambda x: x.lower().strip(), k))
                k = list(key_sets & set(k))
                combinations = itertools.combinations(k,2)
                c = y - START_YEAR
                try: 
                    for comb in combinations:
                        a = word_dict[comb[0]]
                        b = word_dict[comb[1]]
                        if a > b:
                            b, a = a, b
                        datas[a][b][c] = datas[a][b][c] + 1
                except Exception as e:
                    print(e)
                    pass
    except Exception as e:
        # keywordがないものも存在
        pass

(1233, 1233, 21)


0it [00:00, ?it/s]

CPU times: user 2min 29s, sys: 14.1 s, total: 2min 43s
Wall time: 2min 33s


In [8]:
np.save(DATAPATH+"DBLP/pathes.npy", datas)